In [ ]:
from diffusers import StableDiffusionControlNetImg2ImgPipeline, ControlNetModel, UniPCMultistepScheduler
from diffusers.utils import make_image_grid, load_image
from transformers import pipeline

import torch
import sys
import os
import shutil
import time
import PIL
import requests
import matplotlib.pyplot as plt
import numpy as np
from io import BytesIO

import os
os.putenv('HF_HOME', 'D:/CACHE')
os.putenv('TRANSFORMERS_CACHE', 'D:/CACHE')
os.putenv("HF_DATASETS_CACHE", 'D:/CACHE')

In [ ]:
#DEPTH IMAGE
imag_path = "E:/DEV/PYTHON/NEURAL_STYLE_TRANSFER/TORCH_STABLE_DIFFUSION/riaan.jpg"
image = PIL.Image.open(imag_path).convert("RGB").resize((512,512))

def get_depth_map(image, depth_estimator):
    image = depth_estimator(image)["depth"]
    image = np.array(image)
    image = image[:, :, None]
    image = np.concatenate([image, image, image], axis=2)
    detected_map = torch.from_numpy(image).float() / 255.0
    depth_map = detected_map.permute(2, 0, 1)
    return depth_map

depth_estimator = pipeline("depth-estimation",cache_dir= 'D:/CACHE')
depth_map = get_depth_map(image, depth_estimator).unsqueeze(0).half().to("cuda")

In [ ]:
controlnet = ControlNetModel.from_pretrained("lllyasviel/control_v11f1p_sd15_depth", cache_dir= 'D:/CACHE', torch_dtype=torch.float16, safety_checker=None, use_safetensors=True)
pipe = StableDiffusionControlNetImg2ImgPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", cache_dir= 'D:/CACHE', controlnet=controlnet, safety_checker=None,torch_dtype=torch.float16, use_safetensors=True
)
def disabled_safety_checker(images, clip_input):
    if len(images.shape)==4:
        num_images = images.shape[0]
        return images, [False]*num_images
    else:
        return images, False
controlnet.safety_checker = disabled_safety_checker
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

In [ ]:
images = [image]
output = pipe(
    "lego mini figure", image=image, control_image=depth_map, num_inference_steps = 200, guidance_scale = 12, strength = .8
).images
images += output
# make_image_grid([image, output], rows=1, cols=2)

In [ ]:
# Plot pipeline outputs.
def plot_images(images, labels = None):
    N = len(images)
    n_cols = 5
    n_rows = int(np.ceil(N / n_cols))

    plt.figure(figsize = (20, 5 * n_rows))
    for i in range(len(images)):
        plt.subplot(n_rows, n_cols, i + 1)
        if labels is not None:
            plt.title(labels[i])
        plt.imshow(np.array(images[i]))
        plt.axis(False)
    plt.show()

In [ ]:
plot_images(images)

In [ ]:

print(os.getcwd())
from PIL import Image

if torch.cuda.is_available():
    device_name = torch.device("cuda")
    torch_dtype = torch.float16
else:
    device_name = torch.device("cpu")
    torch_dtype = torch.float32

# model_id = "models/controlnet11Models_tileE.safetensors"
model_id = "runwayml/stable-diffusion-v1-5"
pipe = AutoPipelineForImage2Image.from_pretrained(model_id, safety_checker=None, use_safetensors=True, torch_dtype=torch.float16)
def disabled_safety_checker(images, clip_input):
    if len(images.shape)==4:
        num_images = images.shape[0]
        return images, [False]*num_images
    else:
        return images, False
pipe.safety_checker = disabled_safety_checker

pipe = pipe.to("cuda")
# pipe.enable_model_cpu_offload()


imag_path = "path/to/image"
image = PIL.Image.open(imag_path).convert("RGB").resize((512,512))


In [ ]:
image

In [ ]:
pipe.scheduler.compatibles
from diffusers import DPMSolverMultistepScheduler
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config, use_karras_sigmas = True )

In [ ]:
pipe.scheduler

In [ ]:
# Prompt embeddings to overcome CLIP 77 token limit.
# https://github.com/huggingface/diffusers/issues/2136

def get_prompt_embeddings(
    pipe,
    prompt,
    negative_prompt,
    split_character = ",",
    device = torch.device("cpu")
):
    max_length = pipe.tokenizer.model_max_length
    # Simple method of checking if the prompt is longer than the negative
    # prompt - split the input strings using `split_character`.
    count_prompt = len(prompt.split(split_character))
    count_negative_prompt = len(negative_prompt.split(split_character))

    # If prompt is longer than negative prompt.
    if count_prompt >= count_negative_prompt:
        input_ids = pipe.tokenizer(
            prompt, return_tensors = "pt", truncation = False
        ).input_ids.to(device)
        shape_max_length = input_ids.shape[-1]
        negative_ids = pipe.tokenizer(
            negative_prompt,
            truncation = False,
            padding = "max_length",
            max_length = shape_max_length,
            return_tensors = "pt"
        ).input_ids.to(device)

    # If negative prompt is longer than prompt.
    else:
        negative_ids = pipe.tokenizer(
            negative_prompt, return_tensors = "pt", truncation = False
        ).input_ids.to(device)
        shape_max_length = negative_ids.shape[-1]
        input_ids = pipe.tokenizer(
            prompt,
            return_tensors = "pt",
            truncation = False,
            padding = "max_length",
            max_length = shape_max_length
        ).input_ids.to(device)

    # Concatenate the individual prompt embeddings.
    concat_embeds = []
    neg_embeds = []
    for i in range(0, shape_max_length, max_length):
        concat_embeds.append(
            pipe.text_encoder(input_ids[:, i: i + max_length])[0]
        )
        neg_embeds.append(
            pipe.text_encoder(negative_ids[:, i: i + max_length])[0]
        )

    return torch.cat(concat_embeds, dim = 1), torch.cat(neg_embeds, dim = 1)

In [ ]:
prompt = """enter prompt here""" 
negative_prompt = """BadDream UnrealisticDream, (deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime), text, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck""" 
prompt_embeds, negative_prompt_embeds = get_prompt_embeddings(
    pipe,
    prompt,
    negative_prompt,
    split_character = ",",
    device = device_name
)

In [ ]:
# # prompt = """a green lego ninja, with swords, somersaults over an enemy, rooftop, lego building, background oflego city, cars, new york, ink, line drawing"""
# # negative_prompt = """cartoon, painting, illustration, worst quality, low quality, normal quality, shirt"""
negative_prompt = """"""

# # from compel import Compel
# # compel = Compel(tokenizer=pipe.tokenizer, text_encoder=pipe.text_encoder)
# # conditioning = compel.build_conditioning_tensor(prompt)
# # prompt_embeds = conditioning
# # conditioning_neg = compel.build_conditioning_tensor(negative_prompt)
# # negative_prompt_embeds = conditioning_neg

prompt_embeds, negative_prompt_embeds = get_prompt_embeddings(
    pipe,
    prompt,
    negative_prompt,
    split_character = ",",
    device = device_name
)

In [ ]:
# Set to True to use prompt embeddings, and False to
# use the prompt strings.
use_prompt_embeddings = True
# Seed and batch size.
start_idx = 1
batch_size = 5
# seeds = [(4294967311+ (1e8*i)) for i  in range(start_idx , start_idx + batch_size , 1)]
# seeds = [4294967310+(i**5) for i  in range(start_idx , start_idx + batch_size , 1)]
seeds = [4294967315]

# Number of inference steps.
num_inference_steps = 200


# Guidance scale.
guidance_scale = 10
strength = .85

# Image dimensions - limited to GPU memory.
width  = 512
height = 512


images = []

for count, seed in enumerate(seeds):
    start_time = time.time()

    if use_prompt_embeddings is False:
        new_img = pipe(
            prompt = prompt,
            negative_prompt = negative_prompt,
            image = image,
            width = width,
            height = height,
            guidance_scale = guidance_scale,
            strength = strength,
            num_inference_steps = num_inference_steps,
            num_images_per_prompt = 1,
            generator = torch.manual_seed(seed),
        ).images
    else:
        new_img = pipe(
            prompt_embeds = prompt_embeds,
            negative_prompt_embeds = negative_prompt_embeds,
            image = image,
            width = width,
            height = height,
            guidance_scale = guidance_scale,
            strength = strength,
            num_inference_steps = num_inference_steps,
            num_images_per_prompt = 1,
            generator = torch.manual_seed(seed),
            # Version= "v1.5.1",
            # Sampler= "DPM++ SDE Karras",

        ).images


    images = images + new_img

In [ ]:
# Plot pipeline outputs.
def plot_images(images, labels = None):
    N = len(images)
    n_cols = 5
    n_rows = int(np.ceil(N / n_cols))

    plt.figure(figsize = (20, 5 * n_rows))
    for i in range(len(images)):
        plt.subplot(n_rows, n_cols, i + 1)
        if labels is not None:
            plt.title(labels[i])
        plt.imshow(np.array(images[i]))
        plt.axis(False)
    plt.show()

In [ ]:
# Plot pipeline outputs.N
def plot_images(images, labels = None):
    N = len(images)
    n_cols = 5
    n_rows = int(np.ceil(N / n_cols))

    plt.figure(figsize = (20, 5 * n_rows))
    for i in range(len(images)):
        plt.subplot(n_rows, n_cols, i + 1)
        if labels is not None:
            plt.title(labels[i])
        plt.imshow(np.array(images[i]))
        plt.axis(False)
    plt.show()
plot_images(images, seeds[:len(images)])
images[0]